## Do Guider drifts correlate to ellipticity?

Craig Lage  12-Dec-25

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from lsst.daf.butler import Butler
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId, makeDefaultButler
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils import (
    ConsDbClient,
    getAirmassSeeingCorrection,
    getBandpassSeeingCorrection,
)
from lsst.summit.utils.efdUtils import calcNextDay

In [ ]:
client = makeEfdClient()
instrument = 'LSSTCam'
butler = makeDefaultButler(instrument)
#os.environ["no_proxy"] += ",.consdb"
cdb_client = ConsDbClient('http://consdb-pq.consdb:8080/consdb')

In [ ]:
def calculate_r_squared(x, y, coeffs):
    """
    Calculates the R-squared value for a polynomial fit.
    """
    # Create a polynomial function from the coefficients
    p = np.poly1d(coeffs)

    # Calculate the predicted y values
    yhat = p(x)
    # Calculate the mean of the original y values
    ybar = np.mean(y)

    # Calculate the total sum of squares (SStot)
    ss_tot = np.sum((y - ybar)**2)
    # Calculate the residual sum of squares (SSres) or explained sum of squares (SSreg)
    ss_reg = np.sum((yhat - ybar)**2)
    # R-squared is SSreg / SStot
    r_squared = ss_reg / ss_tot
    
    return r_squared

In [ ]:
def fetch(client, day_obs, seq_min, seq_max):
    edge_chips = (0, 1, 2, 3, 18, 19, 20, 23, 64, 65, 68, 71, 155,
                158,160,161,185,186,187,188, 168, 169, 176, 165,
                123,124,120,117,30,33,27,28)
    visits_query = f'''
    SELECT 
    ccdvisit1_quicklook.psf_sigma,
    ccdvisit1_quicklook.psf_ixx,
    ccdvisit1_quicklook.psf_iyy,
    ccdvisit1_quicklook.psf_ixy,
    ccdvisit1.detector,
    visit1_quicklook.aos_fwhm,
    visit1_quicklook.physical_rotator_angle,
    visit1.visit_id,
    visit1.seq_num,
    visit1.day_obs
    FROM
    cdb_lsstcam.ccdvisit1_quicklook AS ccdvisit1_quicklook,
    cdb_lsstcam.ccdvisit1 AS ccdvisit1,
    cdb_lsstcam.visit1_quicklook AS visit1_quicklook,
    cdb_lsstcam.visit1 AS visit1 
    WHERE 
    ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
    AND ccdvisit1.visit_id = visit1.visit_id 
    AND visit1.visit_id = visit1_quicklook.visit_id
    AND ccdvisit1.detector NOT IN {edge_chips}
    AND visit1.day_obs = {day_obs}
    AND (visit1.seq_num BETWEEN {seq_min} AND {seq_max})
    AND (visit1.img_type = 'science')
    '''
    
    ccdvisits = client.query(visits_query).to_pandas()

    pixel_scale = 0.2
    sig2fwhm = 2 * np.sqrt(2 * np.log(2))
    ccdvisits["psf_fwhm"] = ccdvisits["psf_sigma"] * sig2fwhm * pixel_scale
    ccdvisits["psf_fwhm"] = pd.to_numeric(ccdvisits["psf_fwhm"], errors="coerce")
    ccdvisits["psf_ixx"] = pd.to_numeric(ccdvisits["psf_ixx"], errors="coerce")
    ccdvisits["psf_iyy"] = pd.to_numeric(ccdvisits["psf_iyy"], errors="coerce")
    ccdvisits["psf_ixy"] = pd.to_numeric(ccdvisits["psf_ixy"], errors="coerce")
    ccdvisits["aos_fwhm"] = pd.to_numeric(ccdvisits["aos_fwhm"], errors="coerce")
    denom = ccdvisits["psf_ixx"] + ccdvisits["psf_iyy"]
    denom = denom.replace(0, np.nan)
    
    e1 = (ccdvisits["psf_ixx"] - ccdvisits["psf_iyy"]) / denom
    e2 = (2 * ccdvisits["psf_ixy"]) / denom
    e = np.sqrt(e1**2 + e2**2)
    theta = 0.5 * np.atan2(e2, e1)
    ex = e * np.cos(theta)
    ey = e * np.sin(theta)
    ccdvisits["e_theta"] = theta * 180.0 / np.pi
    ccdvisits["e"] = e
    ccdvisits["ex"] = ex
    ccdvisits["ey"] = ey

    return ccdvisits

In [ ]:
startDay = 20251120
endDay = 20251130

columns = ['dayObs', 'expId', 'seqNum', 'alt_drift', 'az_drift', 'total_drift', 'rot', 'ixx', 'ixy', 
           'iyy', 'dxx', 'dxy', 'dyy']
df = pd.DataFrame(columns=columns)
dayObs = startDay
while dayObs <= endDay:
    try:
        table = fetch(cdb_client, dayObs, 0,1500)
        # json data copied from the summit as it's not in consDB yet
        guiderTable = pd.read_json(f'/project/rubintv/LSSTCam/guiders/sidecar_metadata/dayObs_{dayObs}.json').T
        guiderTable = guiderTable.sort_index()
        print(dayObs, len(guiderTable), len(table))
    except:
        print(f"{dayObs} failed!")
        dayObs = calcNextDay(dayObs)
        continue
    for i in range(1, len(guiderTable)+1):
        try:
            seqNum = int(guiderTable.index[i])
            thisGuider = guiderTable[guiderTable.index == seqNum]
            expId = int(dayObs * 1.0E5 + seqNum)
            alt_drift = float(thisGuider['Alt drift (arcsec total)']) * 30.0
            az_drift = float(thisGuider['Az drift (arcsec total)']) * 30.0
            total_drift = np.sqrt(az_drift**2 + alt_drift**2)
            thisTable = table[table['seq_num'] == seqNum]
            rot = thisTable['physical_rotator_angle'].median()
            rotRad = rot * np.pi / 180.0
            R = [[np.cos(rotRad), -np.sin(rotRad)],
                 [np.sin(rotRad), np.cos(rotRad)]]
            (x_drift, y_drift) = np.dot(R, (-alt_drift, az_drift))
            ixx = thisTable['psf_ixx'].median()
            ixy = thisTable['psf_ixy'].median()
            iyy = thisTable['psf_iyy'].median()
            dxx = x_drift * x_drift
            dxy = x_drift * y_drift
            dyy = y_drift * y_drift
            if np.isnan(rot) or np.isnan(az_drift) or np.isnan(alt_drift):
                continue
            dataList = [int(dayObs), expId, seqNum, alt_drift, az_drift, 
                        total_drift, rot, ixx, ixy, iyy, dxx, dxy, dyy]
            df.loc[len(df)] = dataList
            print(f"{expId} succeeded!")
        except:
            print(f"{expId} failed!")
            continue
    
    print(dayObs, len(df))
    dayObs = calcNextDay(dayObs)
print(len(df))

In [ ]:
seqNum = 254
thisDf = df[df['seqNum'] == seqNum]
print(thisDf)

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(12,5))
plt.suptitle(f"Ellipticity vs guider moments {startDay}-{endDay}")
plt.subplots_adjust(wspace=0.3)
x = df['dxx'].values; y = df['ixx'].values
mask = ~(np.isnan(x) | np.isnan(y))
x = x[mask]; y = y[mask]
coeffs = np.polyfit(x, y, 1)
#r_squared = calculate_r_squared(x, y, coeffs)
#axs[0].text(-0.4, 0.4, f"R-squared = {r_squared:.2f}")
#fit_xs = np.linspace(-0.5,0.5, 100)
#fit_ys = fit_xs * coeffs[0] + coeffs[1]
axs[0].set_title("XX")
axs[0].set_xlabel("Guider moment Dxx")
axs[0].set_ylabel("PSF Ixx")
axs[0].scatter(x, y, s=2)
#axs[0].scatter(thisDf['alt_drift'], thisDf['e_alt'], s=20, color='red')
axs[0].set_xlim(0, 1.0)
axs[0].set_ylim(0,10)
#axs[0].plot(fit_xs, fit_ys, ls='--', color='red')
x = df['dyy'].values; y = df['iyy'].values
mask = ~(np.isnan(x) | np.isnan(y))
x = x[mask]; y = y[mask]
coeffs = np.polyfit(x, y, 1)
r_squared = calculate_r_squared(x, y, coeffs)
#axs[1].text(-0.4, 0.4, f"R-squared = {r_squared:.2f}")
#fit_xs = np.linspace(-0.5,0.5, 100)
#fit_ys = fit_xs * coeffs[0] + coeffs[1]
axs[1].set_title("YY")
axs[1].set_xlabel("Guider moment Dyy")
axs[1].set_ylabel("PSF Iyy")
axs[1].scatter(x, y, s=2)
#axs[1].scatter(thisDf['az_drift'], thisDf['e_az'], s=20, color='red')
axs[1].set_xlim(0, 1.0)
axs[1].set_ylim(0, 10)
#axs[1].plot(fit_xs, fit_ys, ls='--', color='red')
x = df['dxy'].values; y = df['ixy'].values
mask = ~(np.isnan(x) | np.isnan(y))
x = x[mask]; y = y[mask]
coeffs = np.polyfit(x, y, 1)
r_squared = calculate_r_squared(x, y, coeffs)
#axs[1].text(-0.4, 0.4, f"R-squared = {r_squared:.2f}")
#fit_xs = np.linspace(-0.5,0.5, 100)
#fit_ys = fit_xs * coeffs[0] + coeffs[1]
axs[2].set_title("XY")
axs[2].set_xlabel("Guider moment Dxy")
axs[2].set_ylabel("PSF Ixy")
axs[2].scatter(x, y, s=2)
#axs[1].scatter(thisDf['az_drift'], thisDf['e_az'], s=20, color='red')
axs[2].set_xlim(-0.2,0.2)
axs[2].set_ylim(-2.0, 2.0)
#axs[1].plot(fit_xs, fit_ys, ls='--', color='red')
fig.savefig(f"/home/cslage/DATA/Ellipticity_vs_Guider_moments_{startDay}-{endDay}.png")
#fig.savefig(f"/home/cslage/DATA/Ellipticity_vs_Guider_drift_2025121600254.png")


In [ ]:
plt.hist(e_theta_stds, bins=20)

In [ ]:
theta_std_limit = 15.0
filtered_e_theta_stds, filtered_az_drifts, filtered_alt_drifts, filtered_e_azs, filtered_e_alts = map(list, zip(
    *[(a, b, c, d, e) for a, b, c, d, e in zip(e_theta_stds, az_drifts, alt_drifts, e_azs, e_alts) if a < theta_std_limit]
))
len(filtered_e_azs)

In [ ]:
theta_std_limit = 15.0
filtered_df = df[df['e_theta_std'] < theta_std_limit]

fig, axs = plt.subplots(1,2,figsize=(10,5))
plt.suptitle(f"Ellipticity vs guider drift {startDay}-{endDay}\nWith cut for e_theta_std < {theta_std_limit:.1f} degrees")
x = filtered_df['alt_drift'].values; y = filtered_df['e_alt'].values
mask = ~(np.isnan(x) | np.isnan(y))
x = x[mask]; y = y[mask]
coeffs = np.polyfit(x, y, 1)
r_squared = calculate_r_squared(x, y, coeffs)
axs[0].text(-0.4, 0.4, f"R-squared = {r_squared:.2f}")
fit_xs = np.linspace(-0.5,0.5, 100)
fit_ys = fit_xs * coeffs[0] + coeffs[1]
axs[0].set_title("Altitude")
axs[0].set_xlabel("Alt guider drift (arcsec per image)")
axs[0].set_ylabel("Ellipticity component along altitude")
axs[0].scatter(x, y, s=2)
axs[0].set_xlim(-0.5,0.5)
axs[0].set_ylim(-0.5,0.5)
axs[0].plot(fit_xs, fit_ys, ls='--', color='red')
x = filtered_df['az_drift'].values; y = filtered_df['e_az'].values
mask = ~(np.isnan(x) | np.isnan(y))
x = x[mask]; y = y[mask]
coeffs = np.polyfit(x, y, 1)
r_squared = calculate_r_squared(x, y, coeffs)
print(r_squared)
fit_xs = np.linspace(-0.5,0.5, 100)
fit_ys = fit_xs * coeffs[0] + coeffs[1]
axs[1].set_title("Azimuth")
axs[1].set_xlabel("Az guider drift (arcsec per image)")
axs[1].set_ylabel("Ellipticity component along azimuth")
axs[1].scatter(x, y, s=2)
axs[1].set_xlim(-0.5,0.5)
axs[1].set_ylim(-0.5,0.5)
axs[1].plot(fit_xs, fit_ys, ls='--', color='red')
#fig.savefig(f"/home/c/cslage/u/Guider_Mode/data/Ellipticity_vs_Guider_drift_2_{startDay}-{endDay}.png")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,5))
plt.suptitle(f"Ellipticity vs guider drift {startDay}-{endDay}")
x = df['total_drift'].values; y = df['e'].values
mask = ~(np.isnan(x) | np.isnan(y))
x = x[mask]; y = y[mask]
coeffs = np.polyfit(x, y, 1)
r_squared = calculate_r_squared(x, y, coeffs)
ax.text(0.1, 0.3, f"R-squared = {r_squared:.2f}")
fit_xs = np.linspace(0, 1.0, 100)
fit_ys = fit_xs * coeffs[0] + coeffs[1]
ax.set_xlabel("Total guider drift (arcsec per image)")
ax.set_ylabel("Total Ellipticity e")
ax.scatter(x, y, s=2)
ax.set_xlim(0, 1.0)
ax.set_ylim(0, 0.4)
ax.plot(fit_xs, fit_ys, ls='--', color='red')
fig.savefig(f"/home/cslage/DATA/Ellipticity_vs_Guider_drift_3_{startDay}-{endDay}.png")


In [ ]:
theta_std_limit = 15.0
filtered_df = df[df['e_theta_std'] < theta_std_limit]

fig, ax = plt.subplots(1,1,figsize=(5,5))
plt.suptitle(f"Ellipticity vs guider drift {startDay}-{endDay}\nWith cut for e_theta_std < {theta_std_limit:.1f} degrees")
x = filtered_df['total_drift'].values; y = filtered_df['e'].values
mask = ~(np.isnan(x) | np.isnan(y))
x = x[mask]; y = y[mask]
coeffs = np.polyfit(x, y, 1)
r_squared = calculate_r_squared(x, y, coeffs)
ax.text(0.1, 0.3, f"R-squared = {r_squared:.2f}")
fit_xs = np.linspace(0, 1.0, 100)
fit_ys = fit_xs * coeffs[0] + coeffs[1]
ax.set_xlabel("Total guider drift (arcsec per image)")
ax.set_ylabel("Total Ellipticity e")
ax.scatter(x, y, s=2)
ax.set_xlim(0, 0.8)
ax.set_ylim(0, 0.4)
ax.plot(fit_xs, fit_ys, ls='--', color='red')
fig.savefig(f"/home/cslage/DATA/Ellipticity_vs_Guider_drift_4_{startDay}-{endDay}.png")


In [ ]:
fig, axs = plt.subplots(1,2,figsize=(10,5))
plt.suptitle(f"Ellipticity vs guider drift {startDay}-{endDay}")
x = np.array(alt_drifts); y = np.array(i_alts)
mask = ~(np.isnan(x) | np.isnan(y))
x = x[mask]; y = y[mask]
coeffs = np.polyfit(x, y, 1)
r_squared = calculate_r_squared(x, y, coeffs)
axs[0].text(-0.4, 0.4, f"R-squared = {r_squared:.2f}")
fit_xs = np.linspace(-0.5,0.5, 100)
fit_ys = fit_xs * coeffs[0] + coeffs[1]
axs[0].set_title("Altitude")
axs[0].set_xlabel("Alt guider drift (arcsec per image)")
axs[0].set_ylabel("Ellipticity component along altitude")
axs[0].scatter(x, y, s=2)
#axs[0].set_xlim(-0.5,0.5)
#axs[0].set_ylim(-0.5,0.5)
axs[0].plot(fit_xs, fit_ys, ls='--', color='red')
x = np.array(az_drifts); y = np.array(i_azs)
mask = ~(np.isnan(x) | np.isnan(y))
x = x[mask]; y = y[mask]
coeffs = np.polyfit(x, y, 1)
r_squared = calculate_r_squared(x, y, coeffs)
fit_xs = np.linspace(-0.5,0.5, 100)
fit_ys = fit_xs * coeffs[0] + coeffs[1]
axs[1].set_title("Azimuth")
axs[1].set_xlabel("Az guider drift (arcsec per image)")
axs[1].set_ylabel("Ellipticity component along azimuth")
axs[1].scatter(x, y, s=2)
#axs[1].set_xlim(-0.5,0.5)
#axs[1].set_ylim(-0.5,0.5)
axs[1].plot(fit_xs, fit_ys, ls='--', color='red')
fig.savefig(f"/home/c/cslage/u/Guider_Mode/data/Ellipticity_vs_Guider_drift_4_{startDay}-{endDay}.png")


In [ ]:
plt.hist(rots)

In [ ]:
theta_limit = 10.0
filtered_e_thetas, filtered_az_drifts, filtered_alt_drifts, filtered_exs, filtered_eys = map(list, zip(
    *[(a, b, c, d, e) for a, b, c, d, e in zip(e_thetas, az_drifts, alt_drifts, exs, eys) if abs(a) < theta_limit]
))
len(filtered_exs)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(10,5))
plt.suptitle(f"Ellipticity vs guider drift {startDay}-{endDay}\nWith cut for |e_theta| <{ theta_limit:.1f} degrees")
x = np.array(filtered_alt_drifts); y = np.array(filtered_eys)
mask = ~(np.isnan(x) | np.isnan(y))
x = x[mask]; y = y[mask]
coeffs = np.polyfit(x, y, 1)
r_squared = calculate_r_squared(x, y, coeffs)
axs[0].text(-0.4, 0.4, f"R-squared = {r_squared:.2f}")
fit_xs = np.linspace(-0.5,0.5, 100)
fit_ys = fit_xs * coeffs[0] + coeffs[1]
axs[0].set_title("Altitude")
axs[0].set_xlabel("Alt guider drift (arcsec per image)")
axs[0].set_ylabel("Ellipticity component along altitude")
axs[0].scatter(x, y, s=2)
axs[0].set_xlim(-0.5,0.5)
axs[0].set_ylim(-0.5,0.5)
axs[0].plot(fit_xs, fit_ys, ls='--', color='red')
x = np.array(filtered_az_drifts); y = np.array(filtered_exs)
mask = ~(np.isnan(x) | np.isnan(y))
x = x[mask]; y = y[mask]
coeffs = np.polyfit(x, y, 1)
r_squared = calculate_r_squared(x, y, coeffs)
print(r_squared)
fit_xs = np.linspace(-0.5,0.5, 100)
fit_ys = fit_xs * coeffs[0] + coeffs[1]
axs[1].set_title("Azimuth")
axs[1].set_xlabel("Az guider drift (arcsec per image)")
axs[1].set_ylabel("Ellipticity component along azimuth")
axs[1].scatter(x, y, s=2)
axs[1].set_xlim(-0.5,0.5)
axs[1].set_ylim(-0.5,0.5)
axs[1].plot(fit_xs, fit_ys, ls='--', color='red')
fig.savefig(f"/home/c/cslage/u/Guider_Mode/data/Ellipticity_vs_Guider_drift_2_{startDay}-{endDay}.png")

In [ ]:
np.sign(-3)

In [ ]:
plt.hist(df['e_theta'].values)